# 第3章 特徴点・特徴量の抽出

## 3.1 特徴点（キーポイント）
特徴点とは、点群の中でも特徴的なパターンを持ち、他の点と区別が付きやすいランドマークなりうる点のことを指す。  
  
### Harris3D
- Harris Corner Detectorの3Dバージョン
    - 3次元点群処理のオープンソースライブラリであるPoint Cloud Library(PCL)が作成したもの
- 画素値勾配の代わりに点の法線ベクトルを用いてHarris指標を計算している
- 最後にNon Maximum Suppression(NMS)を実施している

In [32]:
import open3d as o3d

"""
この関数は特徴点が見やすいように大きくマーキングする関数
→keypointを中心として、radiusの大きさの球で表示される
→最後にpaintで着色して、保存される

spheresはメッシュの箱みたいなイメージ。ここに+=で足していくことで、pointとtrianglesの情報が入った三角メッシュがここで追加される。
→追加された分だけ、後でメッシュとして表示される

ちなみにsphereは球という意味
"""
# This function is only used to make the keypoints look better on the rendering
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([1.0, 0.75, 0.0])
    return spheres

In [40]:
#上記の関数をシミュレーション
spheres = o3d.geometry.TriangleMesh()
print(spheres)
for keypoint in keypoints.points:
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
    #print(sphere)
    #print(sphere.translate(keypoint))
    spheres += sphere
    #print(spheres)
print(spheres)
spheres.paint_uniform_color([1.0, 0.75, 0.0])

TriangleMesh with 0 points and 0 triangles.
TriangleMesh with 32766 points and 65360 triangles.


TriangleMesh with 32766 points and 65360 triangles.

In [41]:
import sys
import open3d as o3d
import numpy as np
#from keypoints_to_spheres import keypoints_to_spheres  
#これはkeypoints_to_spheresというPythonファイルから取ってきたものだからerrorが出ていた

def compute_harris3d_keypoints(pcd, radius=0.01, max_nn=10, threshold=0.001):
    # 法線ベクトルを計算
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    #近傍点探索のためのkd-treeを作成
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    harris = np.zeros(len(np.asarray(pcd.points)) )
    is_active = np.zeros(len(np.asarray(pcd.points)), dtype=bool)

    #Harris指標を計算
    #各点における近傍点群の共分散を計算して、Harris指標を計算する
    for i in range(len(np.asarray(pcd.points))):
        [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
        pcd_normals = pcd.select_by_index(inds)
        pcd_normals.points = pcd_normals.normals
        [_, covar] = pcd_normals.compute_mean_and_covariance()
        harris[ i ] = np.linalg.det( covar ) / np.trace( covar )
        if (harris[ i ] > threshold):
            is_active[ i ] = True

    #NMS(Non Maximum Suppression)
    for i in range(len(np.asarray(pcd.points))):
        if is_active[ i ]:
            [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
            inds.pop(harris[inds].argmax())
            is_active[inds] = False

    keypoints = pcd.select_by_index(np.where(is_active)[0])
    return keypoints

# main
#filename = sys.argv[1]
filename = "../3rdparty/Open3D/examples/test_data/Bunny.ply"
#filename = "G-PCD/stimuli/D01/cube_D01_L01.ply"
#filename = "G-PCD/stimuli/D01/sphere_D01_L01.ply"
#filename = "G-PCD/stimuli/D01/bunny_D01_L01.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

keypoints = compute_harris3d_keypoints(pcd)
print(keypoints)
print(np.asarray(keypoints.points))

pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([keypoints_to_spheres(keypoints), pcd])
#o3d.visualization.draw_geometries([pcd, keypoints])

Loading a point cloud from ../3rdparty/Open3D/examples/test_data/Bunny.ply
PointCloud with 35947 points.
PointCloud with 43 points.
[[-0.0245507   0.183147   -0.010424  ]
 [-0.013506    0.0486161   0.0481837 ]
 [-0.0505546   0.0490872   0.0373101 ]
 [-0.0244966   0.0509689   0.0431921 ]
 [-0.0205114   0.0512056   0.0456176 ]
 [-0.0285043   0.0545632   0.0363758 ]
 [-0.0274872   0.094655    0.0450969 ]
 [-0.00049849  0.101653    0.0443576 ]
 [-0.00888297  0.175714   -0.027834  ]
 [-0.0384872   0.0337149  -0.0296051 ]
 [-0.0741326   0.180484   -0.0536706 ]
 [-0.022585    0.185119   -0.0131839 ]
 [-0.0646788   0.0378404   0.0247428 ]
 [-0.0401042   0.037954    0.0427598 ]
 [-0.0285615   0.0382432   0.0537594 ]
 [-0.0229243   0.0972075   0.044901  ]
 [-0.0430533   0.034543   -0.0263375 ]
 [-0.0108057   0.0342947  -0.0255969 ]
 [-0.0340151   0.0498932  -0.013349  ]
 [-0.0767777   0.176413   -0.0510191 ]
 [-0.0169852   0.186593   -0.0230311 ]
 [-0.0603904   0.152423    0.00027298]
 [-0.03564

### Intrinsic Shape Signature(ISS)
- 固有値分解を用いて計算する 
- 三次元点の近傍点の集合において、それらの点の重心を用いて、共分散行列を計算する
- そして固有値の大きい順にλ1, λ2, λ3として、最小固有値λ3を顕著度として扱う
    - λ2/λ1, λ3/λ2の固有値の比がある閾値以上となる点は除外する
- 最後にHarris3Dと同様にNMS処理を行い、近傍点群の中で顕著度が最大となる点のみ抽出する    

In [6]:
#ISS(Intrinsic Shape Signature)
import sys
import open3d as o3d
#from keypoints_to_spheres import keypoints_to_spheres

#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/Bunny.ply"
filename = "G-PCD/stimuli/D01/cube_D01_L01.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd, salient_radius=0.005, non_max_radius=0.005, gamma_21=0.5, gamma_32=0.5)
print(keypoints)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))
pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([keypoints_to_spheres(keypoints), pcd])

Loading a point cloud from G-PCD/stimuli/D01/cube_D01_L01.ply
PointCloud with 27541 points.
PointCloud with 0 points.


## 3.2 大域特徴量
#### 物体認識
- 一般物体認識
    - 物体のカテゴリ名称を推定する処理
- 特定物体認識
    - 特定のデータベース登録物体のどれに合致するかを推定する処理  
    
#### 特徴量
- 大域特徴量とは、入力データ全体を表現する数値列
- 局所特徴量とは、入力データの局所領域を表現する数値列
    - 多くの場合は、あるデータと別のデータ上の特徴点の対応づけを行うために使用される
    
#### 三次元物体の回転への対処方法
1. 回転不変な特徴量を扱う
    - Spherical Harmonics Representation
2. 主成分分析を行うなどして、物体の基準軸を求め、基準姿勢に回転させる
    - Spin Image
3. 大量に回転させてあらゆる姿勢を試す
    - LightField Descriptor(LFD)

### 3D Shape Histogram
最も基本的な三次元特徴量  
- 多くの三次元特徴量が空間分割によるヒストグラム計算のアプローチ手法をとる  
  
3Dデータの中心を原点として、物体表面上の点を球上等間隔にサンプリングする  
- Shellモデル
    - 球の半径方向の分割
    - 原点を中心として、各点を回転させる変化を加えたときに、変換後の点が変換前の点と同じ領域に含まれるため、物体の回転に対して不変な特徴量が得られる
- Sectorモデル
    - 角度方向の分割
- Combinedモデル
    - 二つのモデルの組み合わせ

### Spherical Harmonics Representation
任意の物体の三次元形状を球面上のある関数とみなし、これを有限個の基底ベクトルの線型結合で近似するもの  
- 任意の関数は、球面調和関数で表すことができる
    - これは三次元形状のフーリエ変換みたいなもの
        - 低周波数の関数は大域的な形状を捉えている
        - 高周波数の関数は局所的な形状を捉えている
- 各周波数における関数の持つエネルギーを並べた特徴量SHとして定義している
    -　この特徴量は回転不変性
- これで最終的に二次元ヒストグラムが得られる    

### LightField Descriptor(LFD)
- 物体を仮想的に配置したカメラにより他方向から見た二次元画像の集まりとして表現する
    - これを多視点画像という
    - 正十二面体の20個の頂点上に配置する
        - 正十二面体の各頂点は3つの辺と接しているから、カメラの位置を別の位置に移動させるようなパターンは20*3=60通りになる
    - さらにN通りの角度から仮想的に撮影して、最後にレンダリングする
- 各カメラ画像からシルエットの輪郭を表現する記述子を計算する
    - あらゆる姿勢の組み合わせにおいてシルエットの類似度の総和を計算する
        - N*(N-1)*60通り

## 3.3 局所特徴量
局所特徴量とは、ある特徴点p周りの局所領域を記述する特徴量

### Spin Image
点群から得られる局所特徴量として最もよく知られている方法。以下のように、特徴点近傍の物理空間を分割する。色付き点群の場合でもうまく行く。  
多くの場合は、特徴点の接平面の法線ベクトルを基準とすることで回転不変性を確保している  
  
全体的な流れ
- ある点pの法線ベクトルnを求める
- その点pの接平面に対してある近傍点qを計算する
- 二つの距離αとβを求めて、二次元ヒストグラムを作成する

### SHOT
点群から抽出する特徴量として、よく知られている手法。特徴点周辺の局所領域を空間分割して、ヒストグラムを求めるアプローチ。  
  
- 特徴点pの近傍点群の共分散行列Mpを固有値分解する手法
    - Mpの固有値分解で得られる基準軸はノイズの影響を受けやすい
    - SHOTではよりノイズに頑健で再現性の高いRFを求めるためのMp'が提案されている
- RFを用いることで、局所空間は半径、方位角、高度の3方向に分割できる
    - 半径２分割、方位角8分割、高度２分割の計32分割して、各領域の法線ベクトルとその領域内の点の法線ベクトルの内積cosθに関するヒストグラムを計算する
  
Spin Imageとの大きな違い
- 空間の基準軸の取り方
    - 特徴点の接平面の法線ベクトルという1軸を基準に取り、Reference Axis(RA)として特徴量を計算する
        - Spin Image
    - 3軸を固定されたReference Frame(RF)に基づいて、特徴量を計算する
        - SHOT

### Fast Point Feature Histograms(FPFH)
#### Point Feature Histograms(PFH)
- ある特徴点pを中心とした小球領域に含まれるk個の近傍点を求め、それらの点から2点を選ぶ全ての組み合わせに対して、計算されるパラメータのヒストグラムを計算する
    - このヒストグラムをSimplified Point Feature Histogram(SPFH)と呼ぶ
- PFH計算量はO(k^2)
これらを用いてFPFHは計算される
- FPFHの計算量はO(k)
    - PFHより高速な処理になっている

In [18]:
#Open3DにはFPFHのみ実装されている
#FPFH(Fast Point Feature Histogram)

import sys 
import open3d as o3d

#filename = sys.argv[1]
filename = "../3rdparty/Open3D/examples/test_data/Bunny.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

#法線ベクトルを計算
pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))
#FPFHを計算
fpth = o3d.pipelines.registration.compute_fpfh_feature(pcd, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))

print(fpth)
print(fpth.data)

Loading a point cloud from ../3rdparty/Open3D/examples/test_data/Bunny.ply
PointCloud with 35947 points.
Feature class with dimension = 33 and num = 35947
Access its data via data member.
[[6.58957766e+00 3.79393273e+01 1.98728631e+01 ... 1.37361634e+01
  5.62345957e+01 6.91576122e+01]
 [4.35214343e-01 9.17400623e+00 3.29149924e+00 ... 1.13916505e+01
  8.43037236e+00 4.78908865e+00]
 [0.00000000e+00 0.00000000e+00 1.23328722e-02 ... 2.90852440e+01
  5.58364644e-01 0.00000000e+00]
 ...
 [1.57724176e-01 6.55616862e+00 3.56026334e-02 ... 1.54064573e+01
  7.06264438e+00 2.16854319e-01]
 [0.00000000e+00 1.57400234e-03 0.00000000e+00 ... 9.37837028e+00
  4.98573232e-01 2.37804435e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.85007116e+00
  2.16728379e-01 0.00000000e+00]]
